## What is visualization?

 Informally, visualization is the transformation of data or information into pictures. Visualization engages the primary human sensory apparatus, vision, as well as the processing power of the human mind. The result is a simple and effective medium for communicating complex and/or voluminous information.

What the computer represents as a series of numbers, we see as a cross section through the human body: skin, bone, and muscle. Even more impressive results are possible when we extend these techniques into three dimensions. Image slices can be gathered into volumes and the volumes can be processed to reveal complete anatomical structures. Using modern techniques, we can view the entire brain, skeletal system, and vascular system on a living patient without interventional surgery. Such capability has revolutionized modern medical diagnostics, and will increase in importance as imaging and visualization technology matures.

Many early uses of visualization were in the engineering and scientific community. From its inception the computer has been used as a tool to simulate physical processes such as ballistic trajectories, fluid flow, and structural mechanics. As the size of the computer simulations grew, it became necessary to transform the resulting calculations into pictures. The amount of data overwhelmed the ability of the human to assimilate and understand it. In fact, pictures were so important that early visualizations were created by manually plotting data. Today, we can take advantage of advances in computer graphics and computer hardware. But, whatever the technology, the application of visualization is the same: to display the results of simulations, experiments, measured data, and fantasy; and to use these pictures to communicate, understand, and entertain.

Visualization is a necessary tool to make sense of the flood of information in today’s world of computers. Without visualization, most of this data would sit unseen on computer disks and tapes. Visualization offers some hope that we can extract the important information hidden within the data. There is another important element to visualization: It takes advantage of the natural abilities of the human vision system. Our vision system is a complex and powerful part of our bodies. We use it and rely on it in almost everything we do. Not only do we have strong 2D visual abilities, but also we are adept at integrating different viewpoints and other visual clues into a mental image of a 3D object or plot. Likewise, we have a talent for recognizing temporal changes in an image. Given an animation consisting of hundreds of frames, we have an uncanny ability to recognize trends and spot areas of rapid change.

With the introduction of computers and the ability to generate enormous amounts of data, visualization offers the technology to make the best use of our highly developed visual senses. Certainly other technologies such as statistical analysis, artificial intelligence, mathematical filtering, and sampling theory will play a role in large-scale data processing. However, because visualization directly engages the vision system and human brain, it remains an unequaled technology for understanding and communicating data.

The output of computer graphics is an image, while the output of visualization is often produced using computer graphics. Sometimes visualization data is in the form of an image, or we wish to visualize object geometry using realistic rendering techniques from computer graphics. 



In [1]:
import pyvista as pv
from pyvista import examples
import nibabel as nib
import numpy as np

# Data representation - image data



In [2]:
file_path = r"copd1_4D.nii.gz"
image_data = nib.load(file_path).get_fdata()
# Extraer datos de la fase inspiratoria
inspiratory_data = image_data[:, :, :, 0]  # Selecciona la primera fase (índice 0)
grid = pv.ImageData()
grid.dimensions = np.array(inspiratory_data.shape)
grid.origin = (1, 1, 1)
grid.spacing = (0.625, 0.625, 2.5)
grid.point_data["values"] = inspiratory_data.flatten(order="F")  # Flatten the array

# Algoritms
methods to transform this data to and from these various representations, eventually generating graphics primitives that we can render. These methods are called algorithms, and are of special interest to those working in the field of visualization. Algorithms are the verbs that allow us to express our data in visual form. By combining these verbs appropriately, we can reduce complex data into simple, readily comprehensible sentences that are the power of data visualization. To describe the various transformations available, we need to categorize algorithms according to the structure and type of transformation. By structure we mean the effects that transformation has on the topology and geometry of the dataset. By type we mean the type of dataset that the algorithm operates on.

Structural transformations can be classified in four ways, depending on how they affect the geometry, topology, and attributes of a dataset.

 - Geometric transformations alter input geometry but do not changed the topology of the dataset. For example, if we translate, rotate, and/or scale the points of a polygonal dataset, the topology does not change, but the point coordinates, and therefore the geometry, does.

 - Topological transformations alter input topology but do not change geometry and attribute data. Converting a dataset type from polygonal data to unstructured grid data, or from image data to unstructured grid, changes the topology but not the geometry. More often, however, the geometry changes whenever the topology does, so topological transformation is uncommon.

 - Attribute transformations convert data attributes from one form to another, or create new attributes from the input data. The structure of the dataset remains unaffected. Computing vector magnitude or creating scalars based on elevation are data attribute transformations.

 - Combined transformations change both dataset structure and attribute data. For example, computing contour lines or surfaces is a combined transformation.

We also may classify algorithms according to the type of data they operate on, or the type of data they generate. By type, we most often mean the type of attribute data, such as scalars or vectors. Typical categories include:

 - Scalar algorithms operate on scalar data. For example, the generation of contour lines of temperature on a weather map.

 - Vector algorithms operate on vector data. Showing oriented arrows of airflow (direction and magnitude) is an example of vector visualization.

 - Tensor algorithms operate on tensor matrices. An example of a tensor algorithm is to show the components of stress or strain in a material using oriented icons.

 - Modelling algorithms generate dataset topology or geometry, or surface normals or texture data. Modelling algorithms tend to be the catch-all category for many algorithms, since some do not fit neatly into any single category mentioned above. For example, generating glyphs oriented according to the vector direction and then scaled according to the scalar value, is a combined scalar/vector algorithm. For convenience we classify such an algorithm as a modelling algorithm, because it does not fit squarely into any other category.


 Algorithms also can be classified according to the type of data they process. This is the most common scheme found in the visualization literature. However, this scheme is not without its problems. Often the categories overlap, resulting in confusion. For example, a category (not mentioned above) is volume visualization, which refers to the visualization of volume data (or in our terminology, image data). This category was initially created to describe the visualization of scalar data arranged on a volume, but more recently, vector (and even tensor) data has been visualized on a volume. Hence, we have to qualify our techniques to volume vector visualization, or other potentially confusing combinations.

## Scalar algorithms

Most algorithms can be written specifically for a particular dataset type, or more generally, treating any dataset type. The advantage of a specific algorithm is that it is usually faster than a comparable general algorithm. An implementation of a specific algorithm also may be more memory efficient and its implementation may better reflect the relationship between the algorithm and the dataset type it operates on. One example of this is contour surface creation. Algorithms for extracting contour surfaces were originally developed for volume data, mainly for medical applications. The regularity of volumes lends itself to efficient algorithms. However, the specialization of volume-based algorithms precludes their use for more general datasets such as structured or unstructured grids. Although the contour algorithms can be adapted to these other dataset types, they are less efficient than those for volume datasets. Our presentation of algorithms favors the more general implementations.

Fundamental algorithms (surface model): 
 - Color map: is a common scalar visualization technique that maps scalar data to colors, and displays the colors on the computer system. The scalar mapping is implemented by indexing into a color lookup table. Scalar values serve as indices into the lookup table.

  - Transfer fuction: A more general form of the lookup table is called a transfer function. A transfer function maps any expression that maps scalar value into a color specification. For example, scalar values into separate intensity values for the red, green, and blue color components. We can also use transfer functions to map scalar data into other information such as local transparency. A lookup table is a discrete sampling of a transfer function. We can create a lookup table from any transfer function by sampling the transfer function at a set of discrete points.
  
  - Contourning: A natural extension to color mapping is contouring. When we see a surface colored with data values, the eye often separates similarly colored areas into distinct regions. When we contour data, we are effectively constructing the boundary between these regions. These boundaries correspond to contour lines (2D) or surfaces (3D) of constant scalar value.Three-dimensional contours are called isosurfaces, and can be approximated by many polygonal primitives.  Examples of isosurfaces include constant medical image intensity corresponding to body tissues such as skin, bone, or other organs. 
    - Since the most common interpolation technique is linear, we generate points on the contour surface by linear interpolation along the edges. If an edge has scalar values 10 and 0 at its two endpoints, and if we are trying to generate a contour line of value 5, then edge interpolation computes that the contour passes through the midpoint of the edge.
    

# Object oriented 

## Surface model

In [3]:
pl = pv.Plotter(shape=[1,3])
_ = pl.add_mesh(grid)
pl.add_text("Color map")
pl.subplot(0, 1)
_ = pl.add_mesh_isovalue(grid)
pl.add_text("isovalue")
pl.subplot(0, 2)

isovalues = grid.contour(np.linspace(0, 2000, 5), method='contour') #'contour', 'marching_cubes' and 'flying_edges'
# Create a new lookup table with oranges
lut = pv.LookupTable()
lut.value_range = (0, 2000)
lut.hue_range = (0.0, 1)
lut.saturation_range = (0.0, 1)
lut.alpha_range = (0.0, 1.0)
lut.scalar_range = (1, 2000)

scalars_rng = (isovalues.active_scalars.min(), isovalues.active_scalars.max())


def make_double_slider(attr, idx):
    """Create a double slider for a given lookup table attribute."""

    def set_min(min_value):
        max_value = getattr(lut, attr)[1]
        if min_value > max_value:
            # force the movement of the maximum value
            max_value = min_value
            pl.slider_widgets[idx * 2 + 1].GetRepresentation().SetValue(max_value)
        setattr(lut, attr, (min_value, max_value))

        if attr == 'scalar_range':
            actor.mapper.scalar_range = getattr(lut, attr)

    def set_max(max_value):
        min_value = getattr(lut, attr)[0]
        if max_value < min_value:
            # force the movement of the minimum value
            min_value = max_value
            pl.slider_widgets[idx * 2].GetRepresentation().SetValue(min_value)
        setattr(lut, attr, (min_value, max_value))

        if attr == 'scalar_range':
            actor.mapper.scalar_range = getattr(lut, attr)

    if attr == 'scalar_range':
        rng = scalars_rng
    else:
        rng = (0, 1)

    # create two overlapping slider bars by hiding the tube of the second
    pl.add_slider_widget(
        set_min,
        rng,
        value=getattr(lut, attr)[0],
        interaction_event='always',
        title=' '.join(attr.split('_')).capitalize(),
        tube_width=0.003,
        pointa=(0.6, 0.9 - 0.165 * idx),
        pointb=(0.9, 0.9 - 0.165 * idx),
    )
    pl.add_slider_widget(
        set_max,
        rng,
        value=getattr(lut, attr)[1],
        interaction_event='always',
        tube_width=0.0,
        pointa=(0.6, 0.9 - 0.165 * idx),
        pointb=(0.9, 0.9 - 0.165 * idx),
    )
actor = pl.add_mesh(isovalues, cmap=lut, lighting=False)
make_double_slider('alpha_range', 0)
make_double_slider('hue_range', 1)
make_double_slider('value_range', 2)
make_double_slider('saturation_range', 3)
make_double_slider('scalar_range', 4)
pl.add_text("5 isovalues")

pl.link_views()
pl.camera_position = [(-grid.center[0], -grid.center[1]*10, grid.center[2]), grid.center, (0,0, -1)]
pl.show()

Widget(value='<iframe src="http://localhost:62496/index.html?ui=P_0x19761fa23f0_0&reconnect=auto" class="pyvis…

## Volume model

In [4]:
pl = pv.Plotter(shape=(2, 2))

pl.add_volume(grid, scalar_bar_args={'title': "No Opacity"})

pl.subplot(0, 1)
pl.add_volume(grid, opacity="linear", scalar_bar_args={'title': "Linear Opacity"})

pl.subplot(1, 0)
pl.add_volume(grid, opacity="sigmoid", scalar_bar_args={'title': "Sigmoidal Opacity"})

pl.subplot(1, 1)
pl.add_volume(grid, opacity="geom_r", scalar_bar_args={'title': "Log Scale Opacity"})
pl.camera_position = [(-grid.center[0], -grid.center[1]*10, grid.center[2]), grid.center, (0,0, -1)]
pl.link_views()
pl.show()

Widget(value='<iframe src="http://localhost:62496/index.html?ui=P_0x1975f3e3230_1&reconnect=auto" class="pyvis…

In [5]:
pl = pv.Plotter()
pl.add_mesh_threshold(grid)
pl.camera_position = [(-grid.center[0], -grid.center[1]*10, grid.center[2]), grid.center, (0,0, -1)]
pl.show()

Widget(value='<iframe src="http://localhost:51913/index.html?ui=P_0x1dc81227750_2&reconnect=auto" class="pyvis…

#
The two visualization algorithms presented thus far, color mapping and contouring, along with opacity alpha values/ transfer function, are simple, effective methods to display scalar information. It is natural to turn to these algorithms first when visualizing data. However, often our data is not in a form convenient to these techniques. The data may not be single-valued (i.e., a scalar), or it may be a mathematical or other complex relationship. That is part of the fun and creative challenge of visualization: We must tap our creative resources to convert data into a form we can visualize. 

Other examples of scalar mapping include an index value into a list of data, computing vector magnitude or matrix determinate, evaluating surface curvature, or determining distance between points. Scalar generation, when coupled with color mapping or contouring, is a simple, yet effective, technique for visualizing many types of data.

# Image oriented rendering

In [7]:
# Download skybox
# cubemap = examples.download_sky_box_cube_map()
cubemap = examples.download_cubemap_space_4k()
grid = grid.contour(isosurfaces= [600], progress_bar= True, method='marching_cubes') #'contour', 'marching_cubes' and 'flying_edges'
pl = pv.Plotter()
pl.add_actor(cubemap.to_skybox())
pl.set_environment_texture(cubemap)  # For reflecting the environment off the mesh
pl.add_mesh(grid, color='ff9999', opacity = 0.5, ambient= 1,diffuse=1,specular= 1, lighting= True, roughness=1.0, metallic=0.0,pbr=True, interpolate_before_map=False, use_transparency=True, render = True)
pl.add_text("generic illumination scene")
pl.camera_position = [(-grid.center[0], -grid.center[1]*10, grid.center[2]), grid.center, (0,0, -1)]
pl.show()

Widget(value='<iframe src="http://localhost:51913/index.html?ui=P_0x1dccc66af90_4&reconnect=auto" class="pyvis…

In [5]:
grid = grid.contour(isosurfaces= [600], progress_bar= True, method='marching_cubes') #'contour', 'marching_cubes' and 'flying_edges'
plotter = pv.Plotter(lighting='none')
plotter.set_background('black')
plotter.add_mesh(grid, color= 'ff9999', opacity = 0.5, ambient= 1,diffuse=1,specular= 1, lighting= True, roughness=0.1, metallic=0.1,pbr=True, interpolate_before_map=True, use_transparency=True)
# floor = pv.Plane(center=(*grid.center[:2], grid.bounds[-1]), i_size=1000, j_size=1000)
# plotter.add_mesh(floor, color='green')

UFO = pv.Light(position=(grid.center[0],grid.center[1],grid.bounds[-2]*(-200)), focal_point=grid.center, color='white')
UFO.positional = True
UFO.cone_angle = 45
UFO.exponent = 10
UFO.intensity = 1
UFO.show_actor()
plotter.add_light(UFO)

UFO2 = pv.Light(position=(grid.center[0],grid.bounds[3] * 2,grid.center[2]), focal_point=grid.center, color='white')
UFO2.positional = True
UFO2.cone_angle = 45
UFO2.exponent = 10
UFO2.intensity = 1
UFO2.show_actor()
plotter.add_light(UFO2)

UFO3 = pv.Light(position=(grid.center[0], - grid.bounds[3] * 2,grid.center[2]), focal_point=grid.center, color='white')
UFO3.positional = True
UFO3.cone_angle = 45
UFO3.exponent = 10
UFO3.intensity = 1
UFO3.show_actor()
plotter.add_light(UFO3)

UFO4 = pv.Light(position=(grid.bounds[1]*2,grid.center[1],grid.center[2]), focal_point=grid.center, color='white')
UFO4.positional = True
UFO4.cone_angle = 45
UFO4.exponent = 10
UFO4.intensity = 1
UFO4.show_actor()
plotter.add_light(UFO4)

UFO5 = pv.Light(position=(-grid.bounds[1]*2,grid.center[1],grid.center[2]), focal_point=grid.center, color='white')
UFO5.positional = True
UFO5.cone_angle = 45
UFO5.exponent = 10
UFO5.intensity = 1
UFO5.show_actor()
plotter.add_light(UFO5)

# enable shadows to better demonstrate lighting
plotter.enable_shadows()
plotter.camera_position = [(-grid.center[0], -grid.center[1]*10, grid.center[2]), grid.center, (0,0, -1)]
plotter.show()

Computing Contour:   0%|          [00:00<?]

Computing Contour: 100%|██████████[00:01<00:00]


Widget(value='<iframe src="http://localhost:61995/index.html?ui=P_0x17055d3cad0_2&reconnect=auto" class="pyvis…